In [1]:
import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from sklearn.decomposition import PCA
from linear_probe import get_linear_acc
import random
import os
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve

In [2]:
data_val=ReadData('/home/hallscope/output/pope_val').read_all()
data_train=ReadData('/home/hallscope/output/pope_train').read_all()
random.shuffle(data_train)

In [3]:
df_train=pd.DataFrame(data_train)
df_test=pd.DataFrame(data_val)

In [4]:
df_train.head()

,img_path,question,label,question_id,category,most_likely
0,/root/autodl-fs/coco_images/train/COCO_train20...,Is there a cup in the image?\nAnswer the quest...,1,14635,popular,{'embedding': [[[ 0.00389099 0.00149536 0.00...
1,/root/autodl-fs/coco_images/train/COCO_train20...,Is there a dining table in the image?\nAnswer ...,0,3530,popular,{'embedding': [[[ 0.00389099 0.00149536 0.00...
2,/root/autodl-fs/coco_images/train/COCO_train20...,Is there a dining table in the image?\nAnswer ...,1,10115,popular,{'embedding': [[[ 0.00389099 0.00149536 0.00...
3,/root/autodl-fs/coco_images/train/COCO_train20...,Is there a toilet in the image?\nAnswer the qu...,1,11137,popular,{'embedding': [[[ 0.00389099 0.00149536 0.00...
4,/root/autodl-fs/coco_images/train/COCO_train20...,Is there a chair in the image?\nAnswer the que...,0,13888,popular,{'embedding': [[[ 0.00389099 0.00149536 0.00...


In [5]:
y_train_val=df_train['label'].to_numpy()
y_test=df_test['label'].to_numpy()

In [6]:
len_train = len(data_train)
train_rate = 0.8

embedding_train_val = df_train['most_likely'].apply(lambda x: np.squeeze(x['embedding'])).to_list()
embedding_train_val = np.stack(embedding_train_val)
embedding_train = embedding_train_val[:int(len_train*train_rate)]
embedding_val = embedding_train_val[int(len_train*train_rate):]

embedding_test = df_test['most_likely'].apply(lambda x: np.squeeze(x['embedding'])).to_list()
embedding_test = np.stack(embedding_test)


y_train = y_train_val[:int(len_train*train_rate)]
y_val = y_train_val[int(len_train*train_rate):]
y_test = y_test


'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''


print(f'trainset length:{len(y_train)}')
print(f'validset length:{len(y_val)}')
print(f'testset length:{len(y_test)}')
print(
    f'trainset postive: {sum(y_train==1)} negative: {sum(y_train==0)}')
print(
    f'testset postive: {sum(y_test==1)} negative: {sum(y_test==0)}')

trainset length:13574
validset length:3394
testset length:3000
trainset postive: 6827 negative: 6747
testset postive: 1500 negative: 1500


In [8]:
%load_ext autoreload 
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCADiscriminator
from ml_utils.metrics import auroc,get_best_split_from_scores

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Grid Search for Best 'n_components' & 'layer'

In [9]:
def evaluator(n_components,i_layer,X,y):
    discriminator=PCADiscriminator(n_components,X[:,i_layer,:])
    scores=discriminator.get_score()
    return auroc(scores,y)

In [10]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,12),
    'layer':range(embedding_val.shape[1])
}
grid_search=GridSearch(evaluator,grid,embedding_val,y_val)
best_paras=grid_search.search()

Grid Searching for best n_components,layer


  0%|          | 0/11 [00:00<?, ?it/s]

found best n_components:1,layer:0 Score: 0.5


/root/miniconda3/envs/dl3.9/lib/python3.9/site-packages/sklearn/decomposition/_pca.py:794: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var


found best n_components:1,layer:2 Score: 0.5129130650345406
found best n_components:1,layer:4 Score: 0.5143549339189752
found best n_components:1,layer:5 Score: 0.5147346839649241
found best n_components:1,layer:17 Score: 0.552877153813361
found best n_components:1,layer:18 Score: 0.8675332472381262
found best n_components:1,layer:21 Score: 0.8886286576131199
found best n_components:1,layer:22 Score: 0.9063539166196756
found best n_components:1,layer:23 Score: 0.9257163048270642
found best n_components:1,layer:27 Score: 0.9337324009479506
found best n_components:1,layer:28 Score: 0.9396221747621525
found best n_components:1,layer:31 Score: 0.9424465005842173


...

 82%|████████▏ | 9/11 [01:02<00:13,  6.91s/it]/root/miniconda3/envs/dl3.9/lib/python3.9/site-packages/sklearn/decomposition/_pca.py:794: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var


found best n_components:10,layer:24 Score: 0.946849064817739


 91%|█████████ | 10/11 [01:09<00:06,  6.90s/it]/root/miniconda3/envs/dl3.9/lib/python3.9/site-packages/sklearn/decomposition/_pca.py:794: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var


found best n_components:11,layer:24 Score: 0.9469284544659544


100%|██████████| 11/11 [01:16<00:00,  6.95s/it]


In [11]:
best_paras

{'n_components': 11, 'layer': 24, 'best_score': np.float64(0.9469284544659544)}

In [12]:
best_layer=best_paras['layer']

View Results on Training set and Test set

In [14]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_val[:,best_layer,:])
best_split=discriminator.get_best_split(y_val)

In [15]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_train[:,best_layer,:])
scores=discriminator.get_score()
y_train=(scores>best_split)
print(f'train set acc: {accuracy_score(y_train,y_train)} AUROC:{roc_auc_score(y_train,scores)}')

train set acc: 1.0 AUROC:1.0


In [16]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_test[:,best_layer,:])
scores=discriminator.get_score()
preds=(scores>best_split)
print(f'test set acc: {accuracy_score(y_test,preds)} AUROC:{roc_auc_score(y_test,scores)}')

test set acc: 0.842 AUROC:0.9337839999999998


Train Linear Prob to Get Better Results

In [20]:
X_train=embedding_train[:, best_layer,:]
y_train=y_train

X_test=embedding_test[:, best_layer,:]
y_test=y_test

In [18]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [19]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores_test = torch.sigmoid(output).cpu().data.numpy()
pred_test=scores_test>0.5
print(f'test set acc: {accuracy_score(y_test,pred_test)} AUROC:{roc_auc_score(y_test,scores_test)}')

test set acc: 0.79 AUROC:0.836399111111111
